In [1]:
# --- CELL 1: AUTHENTICATED SETUP ---
import os
from google.colab import auth

# 1. Authenticate (Click 'Allow' in the popup)
print("Authenticating User...")
auth.authenticate_user()
print("Authentication successful!")

# 2. Install Libraries
print("\nInstalling libraries...")
!pip install -q -U transformers>=4.46.0 peft>=0.10.0 accelerate>=0.28.0 bitsandbytes>=0.43.1 datasets>=2.18.0 trl>=0.8.6

# 3. Download Data using gsutil (Works with private buckets)
# We use the 'gs://' format which is standard for Google Cloud
bucket_name = "cs230-project"
files = ["toolllama_G123_dfs_train.json", "toolllama_G123_dfs_eval.json"]

print("\nDownloading private data...")
for f in files:
    print(f"Downloading {f}...")
    !gsutil cp gs://{bucket_name}/{f} .

# 4. Verify
if os.path.exists("toolllama_G123_dfs_train.json") and os.path.getsize("toolllama_G123_dfs_train.json") > 0:
    print(f"\n--- SUCCESS: Data Downloaded! ---")
    print(f"Train Size: {os.path.getsize('toolllama_G123_dfs_train.json') / 1024**2:.2f} MB")
    print("!!! PLEASE RESTART SESSION NOW (Runtime -> Restart session) !!!")
else:
    print("\n❌ Error: Download failed. Please check if your account has permission to access this bucket.")

Authenticating User...
Authentication successful!

Installing libraries...

Copying gs://cs230-project/toolllama_G123_dfs_train.json...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

\
Operation completed over 1 objects/1.9 GiB.                                      
Copying gs://cs230-project/toolllama_G123_dfs_eval.json...
\ [1 files][  7.6 MiB/  7.6 MiB]                                                
Operation completed over 1 objects/7.6 MiB.                                      

--- SUCCESS: Data Downloaded! ---
Train Size: 1909.66 MB
!!! PLEASE RESTART SESSION NOW (Runtime -> Restart session) !!!


In [1]:
# --- CELL 2: DATA INSPECTION ---
import json
from datasets import load_dataset

# 1. Load the local files (downloaded in Step 1)
data_files = {
    "train": "toolllama_G123_dfs_train.json",
    "test": "toolllama_G123_dfs_eval.json"
}

print("Loading dataset into memory...")
try:
    # We use the 'json' builder to load local files
    dataset = load_dataset("json", data_files=data_files)
    print(f"\n--- SUCCESS: Dataset Loaded ---")
    print(f"Train Rows: {len(dataset['train'])}")
    print(f"Eval Rows: {len(dataset['test'])}")

    # 2. Inspect the Structure
    # This is critical for writing the training loop formatter
    sample = dataset['train'][0]
    print("\n--- RAW DATA SAMPLE ---")
    print(json.dumps(sample, indent=2))

    print("\n--- AVAILABLE KEYS ---")
    print(sample.keys())

except Exception as e:
    print(f"\n❌ Error loading data: {e}")
    print("Check if the files exist in the file browser on the left.")

Loading dataset into memory...


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]


--- SUCCESS: Dataset Loaded ---
Train Rows: 187542
Eval Rows: 762

--- RAW DATA SAMPLE ---
{
  "id": "Step 7: I'm conducting a research project on social media influencers, and I'm particularly interested in gathering data from Instagram. Can you help me retrieve user information for a specific username, such as 'nike'? Additionally, I would like to access their recent media posts. Thank you!",
  "conversations": [
    {
      "from": "system",
      "value": "You are AutoGPT, you can use many tools(functions) to do the following task.\nFirst I will give you the task description, and your task start.\nAt each step, you need to give your thought to analyze the status now and what to do next, with a function call to actually excute your step. Your output should follow this format:\nThought:\nAction\nAction Input:\n\nAfter the call, you will get the call result, and you are now in a new state.\nThen you will analyze your status now, then decide what to do next...\nAfter many (Thought-cal

In [3]:
# --- CELL 2.5: RE-LOGIN ---
from huggingface_hub import notebook_login

print("Please paste your Hugging Face token again.")
print("(This is required because the session restart wiped your credentials.)")
notebook_login()

Please paste your Hugging Face token again.
(This is required because the session restart wiped your credentials.)


In [4]:
# --- CELL 3: OFFICIAL TOOLBENCH BASELINE SFT ---
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# 1. Load Model (L4 Optimized)
print("Loading Llama-3.2-3B-Instruct...")
model_id = "meta-llama/Llama-3.2-3B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto", use_cache=False)
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# 2. Add LoRA Adapters
peft_config = LoraConfig(
    r=16, lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# 3. The Formatter (Tailored to your Data Sample)
def format_toolbench(example):
    # We skip the massive 'system' prompt in the data because it consumes too much context.
    # We replace it with a concise system instruction.
    text = "System: You are AutoGPT, capable of using tools to solve problems.\n"

    turns = example['conversations']
    for turn in turns:
        role = turn['from']
        content = turn['value']

        if role == 'user':
            text += f"User: {content}\n"
        elif role == 'assistant':
            # Contains 'Thought:', 'Action:', 'Action Input:'
            text += f"Assistant: {content}\n"
        elif role == 'function':
            text += f"Tool Output: {content}\n"

    return text + tokenizer.eos_token

# 4. Tokenize & Filter
print("\nTokenizing data (processing 2,000 samples)...")
def tokenize_fn(examples):
    texts = [format_toolbench({"conversations": c}) for c in examples['conversations']]
    return tokenizer(texts, truncation=True, max_length=1024, padding="max_length")

# We select a subset of 2,000 for the project demonstration to ensure it finishes quickly.
# (The full dataset would take hours/days)
train_data = dataset['train'].select(range(2000)).map(tokenize_fn, batched=True)
eval_data = dataset['test'].select(range(100)).map(tokenize_fn, batched=True)

# 5. Train
print("\nStarting Training...")
trainer = Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    args=TrainingArguments(
        output_dir="./toolllama_sft_official",
        max_steps=100,                   # 100 steps is enough to prove learning
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        logging_steps=10,
        fp16=True,
        optim="paged_adamw_8bit",
        report_to="none"
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
)

trainer.train()
trainer.save_model("./toolllama_sft_final_adapter")
print("\n--- 🚀 OFFICIAL TRAINING COMPLETE 🚀 ---")

Loading Llama-3.2-3B-Instruct...


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

...

Map:   0%|          | 0/100 [00:00<?, ? examples/s]


Starting Training...


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.630400
20,1.285300
30,1.041400
40,0.957500
50,1.017900
60,0.894000
70,0.947500
80,0.869700
90,0.957200
100,0.907600



--- 🚀 OFFICIAL TRAINING COMPLETE 🚀 ---


In [6]:
# --- CELL 1: INSTALL & DOWNLOAD DATA ---
import os
from google.colab import auth

# 1. Install Dependencies
print("Installing libraries...")
!pip install -q -U transformers>=4.46.0 peft>=0.10.0 accelerate>=0.28.0 bitsandbytes>=0.43.1 datasets>=2.18.0 trl>=0.8.6

# 2. Authenticate for GCS (Click 'Allow' in the popup)
print("\nAuthenticating to download data...")
auth.authenticate_user()

# 3. Download Datasets
print("\nDownloading ToolBench data...")
bucket_name = "cs230-project"
files = ["toolllama_G123_dfs_train.json", "toolllama_G123_dfs_eval.json"]

for f in files:
    if not os.path.exists(f):
        print(f"Downloading {f}...")
        !gsutil cp gs://{bucket_name}/{f} .
    else:
        print(f"{f} already exists.")

print("\n--- SUCCESS: Environment Ready ---")
print("Please RESTART SESSION (Runtime -> Restart session) before running the next cell.")

Installing libraries...

Authenticating to download data...

toolllama_G123_dfs_train.json already exists.
toolllama_G123_dfs_eval.json already exists.

--- SUCCESS: Environment Ready ---
Please RESTART SESSION (Runtime -> Restart session) before running the next cell.


restart

In [ ]:
# --- CELL 2: KNOWLEDGE DISTILLATION (KD) TRAINING ---
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset
from huggingface_hub import notebook_login

# 0. Login (Required for Llama models)
notebook_login()

# 1. Configuration
teacher_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
student_id = "meta-llama/Llama-3.2-3B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

# 2. Load Data
print("Loading ToolBench Data...")
dataset = load_dataset("json", data_files={"train": "toolllama_G123_dfs_train.json", "test": "toolllama_G123_dfs_eval.json"})

# 3. Load TEACHER (Frozen, 8B)
print(f"Loading Teacher: {teacher_id}...")
teacher_model = AutoModelForCausalLM.from_pretrained(
    teacher_id,
    quantization_config=bnb_config,
    device_map={"": 0}, # Force to same GPU
    use_cache=False
)
teacher_model.eval()

# 4. Load STUDENT (Trainable, 3B)
print(f"Loading Student: {student_id}...")
tokenizer = AutoTokenizer.from_pretrained(teacher_id) # Use teacher's tokenizer for compatibility
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    student_id,
    quantization_config=bnb_config,
    device_map={"": 0},
    use_cache=False
)
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# 5. Attach LoRA to Student
peft_config = LoraConfig(
    r=16, lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)

# 6. Define Custom Distillation Trainer
class DistillationTrainer(Trainer):
    def __init__(self, teacher_model=None, temperature=2.0, alpha=0.5, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher = teacher_model
        self.temperature = temperature
        self.alpha = alpha
        self.kl_loss = nn.KLDivLoss(reduction="batchmean")

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        # Student forward pass
        student_outputs = model(**inputs)

        # Teacher forward pass (No Grad)
        with torch.no_grad():
            teacher_outputs = self.teacher(**inputs)

        # Distillation Loss (KL Divergence)
        s_logits = student_outputs.get("logits")
        t_logits = teacher_outputs.get("logits")

        s_soft = F.log_softmax(s_logits / self.temperature, dim=-1)
        t_soft = F.softmax(t_logits / self.temperature, dim=-1)

        distill_loss = self.kl_loss(s_soft, t_soft) * (self.temperature ** 2)
        standard_loss = student_outputs.loss

        # Combined Loss
        total_loss = (self.alpha * standard_loss) + ((1 - self.alpha) * distill_loss)

        return (total_loss, student_outputs) if return_outputs else total_loss

# 7. Formatting & Tokenization
def format_toolbench(example):
    text = "System: You are a helpful assistant that can use tools to solve problems.\n"
    turns = example.get('conversations', [])
    for turn in turns:
        role = turn.get('from', '')
        content = turn.get('value', '')
        if role == 'human': text += f"User: {content}\n"
        elif role == 'gpt': text += f"Assistant: {content}\n"
        elif role == 'function': text += f"Tool Output: {content}\n"
    return text + tokenizer.eos_token

print("Tokenizing data (2000 samples)...")
def tokenize_fn(examples):
    texts = [format_toolbench({"conversations": c}) for c in examples['conversations']]
    return tokenizer(texts, truncation=True, max_length=1024, padding="max_length")

train_data = dataset['train'].select(range(2000)).map(tokenize_fn, batched=True)

# 8. Start Distillation Training
print("Starting Knowledge Distillation...")
training_args = TrainingArguments(
    output_dir="./toolllama_kd_run",
    max_steps=100,
    per_device_train_batch_size=1, # Keep batch size 1 for safety with 2 models
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    logging_steps=10,
    fp16=True,
    optim="paged_adamw_8bit",
    report_to="none",
    remove_unused_columns=False
)

trainer = DistillationTrainer(
    teacher_model=teacher_model,
    model=model,
    args=training_args,
    train_dataset=train_data,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    temperature=2.0,
    alpha=0.5
)

trainer.train()
trainer.save_model("./toolllama_kd_adapter")
print("\n--- 🎓 DISTILLATION COMPLETE 🎓 ---")

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Loading ToolBench Data...
